# Introduction

This tutorial gives a short overview of the AD-module included in PorePy.

In [1]:
import numpy as np
import scipy.sparse as sps

from porepy.ad.forward_mode import Ad_array

# Declaring an AD-variable

We initiate a variable $x =2$ by giving a pair (val, jac) to the Ad_array class. val is the value at which the function will be evaluated and jac =1 since $\frac{d x}{dx} = 1$. We can further declare a constant $c=3$ using the pair (3, 0).

In [3]:
x = Ad_array(2, 1)
c = Ad_array(3, 0)

We now define a function $y=x^2 + 3$ 

In [4]:
y = x**2 + c

To obtain the function value and the derivative we can call .val and .jac

In [6]:
print('y value is: ', y.val)
print('dy/dx is: ', y.jac)

y value is:  7
dy/dx is:  4.0
